In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from ast import literal_eval

import json

In [3]:
spark = SparkSession.builder.appName("Test").getOrCreate()

22/05/05 09:54:42 WARN Utils: Your hostname, david-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.86.64 instead (on interface wlp43s0)
22/05/05 09:54:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 09:54:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
## reading the dataset
df = spark.read.csv('../../files/',header=True,inferSchema=True)

In [7]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
df.show()

+--------------------+------------+--------------------+
|           match_url|      bookie|                odds|
+--------------------+------------+--------------------+
|https://www.oddsp...|    10x10bet|['17/100', '71/10...|
|https://www.oddsp...|       1xBet|['9/50', '37/5', ...|
|https://www.oddsp...| bet-at-home|['4/25', '31/5', ...|
|https://www.oddsp...|      bet365|['7/50', '13/2', ...|
|https://www.oddsp...|    BetFinal|['9/50', '13/2', ...|
|https://www.oddsp...|        bwin|['4/25', '7/1', '...|
|https://www.oddsp...|     Coolbet|['9/50', '143/20'...|
|https://www.oddsp...|     Curebet|['17/100', '71/10...|
|https://www.oddsp...|     Ditobet|['17/100', '37/5'...|
|https://www.oddsp...|       GGBET|['9/50', '337/50'...|
|https://www.oddsp...|      Lasbet|['17/100', '37/5'...|
|https://www.oddsp...| Marathonbet|['17/100', '37/5'...|
|https://www.oddsp...|      N1 Bet|['17/100', '172/2...|
|https://www.oddsp...|   Parimatch|['3/20', '69/10',...|
|https://www.oddsp...|    Pinna

In [10]:
def ConvertProbabilityToDecimal(probability_list):
    decimal_list = []
    probability_list = literal_eval(probability_list)
    for i in probability_list:
        num,den = i.split("/")
        decimal_odd = 1 + float(num) / float(den)
        
        decimal_list.append(decimal_odd)
        
    return decimal_list

In [11]:
convertOddsUDF = udf(lambda z: ConvertProbabilityToDecimal(z), StringType())

In [16]:
df = df.withColumn("decimal_odds", convertOddsUDF(df.odds))

In [17]:
df.show(100)

+--------------------+------------+--------------------+--------------------+
|           match_url|      bookie|                odds|        decimal_odds|
+--------------------+------------+--------------------+--------------------+
|https://www.oddsp...|    10x10bet|['17/100', '71/10...|   [1.17, 8.1, 19.0]|
|https://www.oddsp...|       1xBet|['9/50', '37/5', ...|   [1.18, 8.4, 18.0]|
|https://www.oddsp...| bet-at-home|['4/25', '31/5', ...|   [1.16, 7.2, 15.0]|
|https://www.oddsp...|      bet365|['7/50', '13/2', ...|[1.14000000000000...|
|https://www.oddsp...|    BetFinal|['9/50', '13/2', ...|   [1.18, 7.5, 17.0]|
|https://www.oddsp...|        bwin|['4/25', '7/1', '...|   [1.16, 8.0, 16.5]|
|https://www.oddsp...|     Coolbet|['9/50', '143/20'...|  [1.18, 8.15, 19.5]|
|https://www.oddsp...|     Curebet|['17/100', '71/10...|   [1.17, 8.1, 19.0]|
|https://www.oddsp...|     Ditobet|['17/100', '37/5'...|   [1.17, 8.4, 21.0]|
|https://www.oddsp...|       GGBET|['9/50', '337/50'...| [1.18, 